In [347]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [348]:
data = pd.read_excel("test.xlsx")
data.head()

,hour,date_miladi,date_shamsi,code,unit_no,fuel_type,mvar,temp,moisture,power
0,1,2020-01-13,1398/10/23,SO,1,A,11,3,94,119
1,2,2020-01-13,1398/10/23,SO,1,A,11,3,96,119
2,3,2020-01-13,1398/10/23,SO,1,A,10,2,95,120
3,4,2020-01-13,1398/10/23,SO,1,A,11,2,95,120
4,5,2020-01-13,1398/10/23,SO,1,A,11,2,95,121


In [349]:
data=data[data["power"]>100]
data.shape

(756, 10)

In [350]:
# Change object to integer:
data["fuel_type"][data["fuel_type"]=="A"] = 1; 
data["fuel_type"][data["fuel_type"]=="B"] = 2;
data["fuel_type"][data["fuel_type"]=="C"] = 3;

<ipython-input-350-6b0e21f41b39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["fuel_type"][data["fuel_type"]=="B"] = 2;
<ipython-input-350-6b0e21f41b39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["fuel_type"][data["fuel_type"]=="C"] = 3;


In [351]:
data['fuel_type'].value_counts()

1    508
Name: fuel_type, dtype: int64

In [352]:
data["fuel_type"].fillna(method='bfill', inplace=True)

In [353]:
data.isnull().sum()

hour           0
date_miladi    0
date_shamsi    0
code           0
unit_no        0
fuel_type      0
mvar           0
temp           0
moisture       0
power          0
dtype: int64

In [354]:
X = data[["moisture","unit_no","temp","fuel_type"]]
y = data[["power"]]
y

,power
0,119
1,119
2,120
3,120
4,121
...,...
994,110
995,127
996,127
997,124


In [355]:
X

,moisture,unit_no,temp,fuel_type
0,94,1,3,1
1,96,1,3,1
2,95,1,2,1
3,95,1,2,1
4,95,1,2,1
...,...,...,...,...
994,6,6,15,1
995,8,6,16,1
996,21,6,18,1
997,11,6,19,1


In [356]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [357]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
y_train = np.array(y_train)

In [358]:
# Apply PCA to reduce the number of features to 3
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)

# Linear regression without using pca

In [359]:
model_without_pca = LinearRegression()
model_without_pca.fit(X_train, y_train)

LinearRegression()

In [360]:
# assume row is the new data point to predict with 4 features
row = np.array([45, 5, 100 ,1])
X_test_scaled = scaler.transform([row])
y_pred = model_without_pca.predict(X_test_scaled)
print(y_pred)

[[116.21663951]]


# Linear regression with using pca

In [361]:
model_pca = LinearRegression()
model_pca.fit(X_train_pca, y_train)

LinearRegression()

In [362]:
# assume row is the new data point to predict with 3 features
row = np.array([94, 1, 100 ,1])

X_test_scaled = scaler.transform([row])
X_test_pca = pca.transform(X_test_scaled)
y_pred = model_pca.predict(X_test_pca)


print(y_pred)

[[117.06900429]]


# Decision Tree Regressor

In [363]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [364]:
# Training the Decision Tree Regression model on the whole dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)
y_pred=regressor.predict([[94, 1, 100 ,1]])
y_pred[0]

111.0

In [365]:
from sklearn.linear_model import Lasso  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [372]:
model = Lasso(alpha=0.1 , normalize=True)
model.fit(X_train , y_train)
y_pred=model.predict([[94, 1, 100 ,1]])
y_pred[0]

119.56366331647801